# Import Libraries

The FitnessGram™ Pacer Test is a multistage aerobic capacity test that progressively gets more difficult as it continues. The 20 meter pacer test will begin in 30 seconds. Line up at the start. The running speed starts slowly, but gets faster each minute after you hear this signal. <i>beep</i> A single lap should be completed each time you hear this sound. <i>ding</i> Remember to run in a straight line, and run as long as possible. The second time you fail to complete a lap before the sound, your test is over. The test will begin on the word start. <b>On your mark, get ready, start.</b>

In [7]:
import ase.io # for reading *.xyz
import pandas as pd # for dataframes?
import numpy as np # for stats and such
import matplotlib.pyplot as plt # for plotting error distribution

# Setting Variables

In [8]:
DATASET_LOCATION = "./data_structures_errors_head.xyz"

# Reading and Importing Dataset

The ASE does not read the energy, it only reads the atom and atom position?

In [9]:
with open(DATASET_LOCATION, "r") as file:
    for i in file.readlines():
        if "Lattice" in i:
            print(i)

Lattice="11.832514762878418 0.0 -0.02639799937605858 -3.048229455947876 16.438873291015625 -1.1482430696487427 0.0 0.0 31.385263442993164" Properties=species:S:1:pos:R:3 energy=0.3250738802633464 pbc="T T T"

Lattice="12.750493049621582 0.0 0.0 3.1876232624053955 13.42756175994873 -1.8403751850128174 0.0 0.0 29.446002960205078" Properties=species:S:1:pos:R:3 energy=-1.4864445447367416 pbc="T T T"

Lattice="11.985345840454102 0.0 0.0 0.0 7.051717758178711 -3.995115280151367 0.0 0.0 35.95603561401367" Properties=species:S:1:pos:R:3 energy=0.19834090526336468 pbc="T T T"

Lattice="8.023002624511719 0.0 -3.6558828353881836 4.537977695465088 14.066129684448242 -0.6725652813911438 0.0 0.0 42.52546310424805" Properties=species:S:1:pos:R:3 energy=-5.876447284736628 pbc="T T T"

Lattice="11.047003746032715 0.0 0.0 5.523501873016357 9.566986083984375 4.357623100280762 0.0 0.0 34.860984802246094" Properties=species:S:1:pos:R:3 energy=-2.1032161647366365 pbc="T T T"



In [10]:
test = ase.io.iread(filename=DATASET_LOCATION, format="xyz")
for count, item in enumerate(test): 
    print(count, item.todict())

0 {'numbers': array([13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 13, 13, 13, 13, 13, 45, 45, 45, 45, 45, 45, 45, 45,
       45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45,  7,
        1]), 'positions': array([[ 7.02555561,  0.        , 12.73458958],
       [ 6.96548843, 10.95924854, 17.1955719 ],
       [ 1.07926452,  5.47962427, 14.97827911],
       [-1.92391455, 13.69906044, 18.330616  ],
       [ 4.02237654,  8.21943665, 16.08692551],
       [ 9.96866703,  2.73981214, 13.84323502],
       [ 3.7433207 , 12.48701859, 14.12751198],
       [ 9.68961143,  7.00739479, 11.88382244],
       [ 0.84525955,  1.52777028, 17.51944542],
       [-2.17380166, 12.64488316, 13.75287628],
       [ 3.77248931,  7.16525888, 11.50918579],
       [ 6.76065159,  1.68563461, 17.11841202],
       [ 